> # Projeto de Otimização Contínua - EMAp/FGV
 Aluna: Tarla Lemos de Andrade <br>
 Prof.:Vincent Guigues <br>
 Data: 15/12/21

## Problema de alocação de veículos
Neste documento estão os códigos referentes ao relatório do problema de alocação de veículos.

Dados das 10 primeiras instâncias:

* 100 veículos;
* 53 terminais;
* 36 períodos de tempo.

In [19]:
from gurobipy import *
import numpy as np
import pandas as pd
from datetime import datetime

In [20]:
i = 1 #instância
dados = np.loadtxt("p"+str(i)+"_data.dat", unpack = True, dtype=int)

In [4]:
num_veiculos = dados[2]
num_terminais = dados[0]
num_periodos = dados[1]

In [23]:
#Carregando informações
p_ij_0 = np.loadtxt("p"+str(i)+"_pro.dat", unpack = True, dtype=int) #Lucro
c_ij_0 = np.loadtxt("p"+str(i)+"_cos.dat", unpack = True, dtype=int) #custo
A_ijv_0 = np.loadtxt("p"+str(i)+"_A.dat", unpack = True, delimiter=";",dtype=int) #Restrição de movimento
m_itv_0 = np.loadtxt("p"+str(i)+"_m.dat", unpack = True, delimiter=";", dtype=int) #Adição de veículos no sistema
d_ijt_0 = np.loadtxt("p"+str(i)+"_dem.dat", unpack = True, delimiter=";", dtype=int) #demanda
tau_ij_0 = np.loadtxt("p"+str(i)+"_dist.dat", unpack = True, dtype=int) #Tempo de viagem

In [7]:
#Arrumando os dados em dicionários 
A_ijv = {(i,j,v): 0 for i in range(1,1+num_terminais) for j in range(1,1+num_terminais) for v in range(1,1+num_veiculos)}
for i,j,v,k in np.transpose(A_ijv_0):
    A_ijv[(i,j,v)]=1
    
p_ij = {(i,j): 0 for i in range(1,1+num_terminais) for j in range(1,1+num_terminais)}
for i in range(p_ij_0.shape[0]):
    for j in range(p_ij_0.shape[1]):
        p_ij[(i+1,j+1)]=p_ij_0[i,j]
        
c_ij = {(i,j): 0 for i in range(1,1+num_terminais) for j in range(1,1+num_terminais)}
for i in range(c_ij_0.shape[0]):
    for j in range(i+1,c_ij_0.shape[1]):
        c_ij[(i+1,j+1)]=c_ij_0[i,j]
        
m_itv = {(i,t,v): 0 for i in range(1,1+num_terminais) for t in range(1,1+num_periodos) for v in range(1,1+num_veiculos)}
for i,t,v,k in np.transpose(m_itv_0):
    m_itv[(i,t,v)]=1
    
d_ijt = {(i,j,t): 0 for i in range(1,num_terminais+1) for j in range(1,num_terminais+1) for t in range(1,1+num_periodos)}
for i,j,t,d in np.transpose(d_ijt_0):
    d_ijt[(i,j,t)]=d
    
tau_ij = {(i,j): 0 for i in range(1,1+num_terminais) for j in range(1,1+num_terminais)}
for i in range(tau_ij_0.shape[0]):
    for j in range(i+1,tau_ij_0.shape[1]):
        tau_ij[(i+1,j+1)]=tau_ij_0[i,j]
    
aux_x_ijtv = {(i,j,t,v):0 for i in range(1,1+num_terminais) for j in range(1,1+num_terminais) for t in range(1,1+num_periodos) for v in range(1,1+num_veiculos)}


In [ ]:
#modelo
m = Model("VAP")
m.modelSense = GRB.MAXIMIZE
print("Modelo construído.")

In [ ]:
#Adicionando variáveis
start_time_vars = datetime.now()
x_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)
print('Variável (1/2) construída.')
y_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)
end_time_vars = datetime.now()
t_vars = end_time_vars-start_time_vars
print('Variável (2/2) construída.')

In [ ]:
#Função objetivo
start_time_obj = datetime.now()
m.setObjective(quicksum(p_ij[(i,j)]*x_ijtv[(i,j,t,v)]-c_ij[(i,j)]*y_ijtv[(i,j,t,v)] for i,j,t,v in aux_x_ijtv))
end_time_obj = datetime.now()
t_obj = end_time_obj-start_time_obj

In [ ]:
#Restrições
start_time_const = datetime.now()
m.addConstrs(quicksum(x_ijtv[(i,j,t,v)]+y_ijtv[(i,j,t,v)] for j in range(1,num_terminais)) 
             -quicksum(x_ijtv[(k,i,(t-tau_ij[k-1,i-1]),v)] + y_ijtv[(k,i,(t-tau_ij[k-1,i-1]),v)] for k in range(1,num_terminais+1) if k!=i and t>tau_ij[k-1,i-1])
             -y_ijtv[(i,i,(t-1),v)] == m_itv[(i,t,v)] for i,t,v in m_itv if t>1)
print('Restrição (1/4) construída.')
m.addConstrs(quicksum(x_ijtv[(i,j,t,v)] for v in range(1,num_veiculos))<=d_ijt[(i,j,t)] for i,j,t in d_ijt)
print('Restrição (2/4) construída.')
m.addConstrs(A_ijv[(i,j,v)]-x_ijtv[(i,j,t,v)]>=0 for i,j,t,v in x_ijtv)
print('Restrição (3/4) construída.')
m.addConstrs(A_ijv[(i,j,v)]-y_ijtv[(i,j,t,v)]>=0 for i,j,t,v in y_ijtv)
end_time_const = datetime.now()
t_const = end_time_const-start_time_const
print('Restrição (4/4) construída.')

In [ ]:
#Otimizando
start_time_opt = datetime.now()
m.optimize()
end_time_opt = datetime.now()
t = end_time_opt-start_time_opt

### Tentativa do método de geração de colunas

Obs: Código incompleto.

In [ ]:
#################################
#  MÉTODO DE GERAÇÃO DE COLUNAS #
#################################

# https://www.youtube.com/watch?v=1HkX2ZBGtTM&ab_channel=decide4AI

### PROBLEMA DE CORTE #####
class MasterProblem:
    def _init_(self, patternDf, inputDf):
        self.patternCost = patternDf["PaternCost"].values
        self.pattern = patternDf["PatternFill"].values
        self.amount = inputDf["Amount"].values
        self.model = gu.Model("MasterProblem")
        self.patternsIndex = patternDf.index.values
        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
        
    def generateVariables(self):
        self.patternUseVar = self.model.addVars(self.patternsIndex, lb = 0, ub = sum(self.amount),
                                               vtype = GRB.INTEGER, name = "PatternUseVar")
        
    def generateConstraints(self):
        for i in range(len(self.patternsIndex)):
            self.model.addConstr(quicksum(self.pattern[p][i]*self.patternUseVar[p] for p in self.patternsIndex) >=
                                 self.amount[i], "C"+str(i))
            
    def generateObjective(self):
        self.model.setObjective(quicksum(self.patternUseVar[p]*self.patternCost[p] for p in self.patternsIndex),
                               sense=GRB.MINIMIZE)
        
    def solveRelaxedModel(self):
        #relax integer variable to continuous variable
        self.relaxedModel = self.model.relax()
        self.relaxedModel.optimize()
        
    def getDuals(self):
        return self.relaxedModel.getAttr("P1", self.model.getConstrs())
    

In [ ]:
class Subproblem:
    def _init_(self, inputDf, rollWidth, duals):
        self.patternCost = patternDf["PaternCost"].values
        self.pattern = patternDf["PatternFill"].values
        self.amount = inputDf["Amount"].values
        self.pieceSize = inputDf["Size"].values
        self.rollWidth = rollWidth
        self.duals = duals
        self.model = gu.Model("MasterProblem")
        self.patternsIndex = patternDf.index.values
        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
        
    def generateVariables(self):
        self.patternUseVar = self.model.addVars(self.patternsIndex, lb = 0, ub = sum(self.amount),
                                               vtype = GRB.INTEGER, name = "PatternUseVar")
        
    def generateConstraints(self):
        for i in range(len(self.patternsIndex)):
            self.model.addConstr(quicksum(self.pattern[p][i]*self.patternUseVar[p] for p in self.patternsIndex) >=
                                 self.amount[i], "C"+str(i))
    def generateObjective(self):
        self.model.setObjective(quicksum(self.patternUseVar[p]*self.patternCost[p] for p in self.patternsIndex),
                               sense=GRB.MINIMIZE)
    

In [ ]:
class InitialPatternsGenerator:
    def __init__(self, nbItems):
        columns = ['PatternCost', 'PatternFill']
        patterns = pd.DataFrame(index = range(nbItems), columns=columns)
        self.patternDf = patterns
        self.nbItems = nbItems
        
    def generateBasicInitialPatterns(self):
        self.patternDF['PatternCost'] = 1
        self.patternDf['PatternFill'] = [np.where(self.patternDf.index==j,1,0) for j in range(nbItems)]
        return self.patternDf

In [ ]:
class InitialAllocationGenerator:
    def __init__(self, nbItems):
        columns = ['PatternCost', 'PatternFill']
        patterns = pd.DataFrame(index = range(nbItems), columns=columns)
        self.patternDf = patterns
        self.nbItems = nbItems
        
    def generateBasicInitialPatterns(self):
        self.patternDF['PatternCost'] = 1
        self.patternDf['PatternFill'] = [np.where(self.patternDf.index==j,1,0) for j in range(nbItems)]
        return self.patternDf

In [ ]:
class subproblem:
    def getNewPattern(self):
        return self.model.getAttr("X", self.model.getVars())
    
class MasterProblem:
    def addColumn(self, objective, newPattern):
        ctName = ("PatternUserVar[%s]" %len(self.model.getVars()))
        newColumn = Column(newPattern, self.model.getConstrs())
        self.model.addVar(vtype = GRB.INTEGER, lb=0, obj=objective, column=newColumn, name=ctName)
        self.model.update()

In [ ]:
def solvemodel(self, timeLimit=None, Gap=epsilon):
    self.model.setParam('TimeLimit', TimeLimit)
    self.model.setParam('MIGap',epsilon)
    self.model.optimize()

In [ ]:
#generate initial patterns
patternGenerator = InitialPatternsGenerator(len(inputDf)) #pode ser len da demanda?
patternDf = patternGenerator.generateBasicInitialPatterns()

#Build Master problem with initial patterns
master = MasterProblem(patternDf, inputDf)
master.buildModel()

modelImprovable = True

while(modelImprovable):
    #solve relaxed  master
    master.solveRelaxedmodel()
    duals = master.getDuals()
    #build subproblem
    subproblem = Subproblem(inputDf, rollWidth, duals)
    subproblem.buildModel()
    subproblem.solveModel(120,0.05)
    #check if new pattern improves Master solution 
    modelImprovable = (subproblem.getObjectiveValue()>0)
    #Add new generated pattern to master problem and iterate
    newPatternCost = 1
    newPatternCuts = subproblem.getNewPattern()
    master.addColumn(newPatternCost, newPatternCuts)
    
    
#solve the integer master problem with all the patternsinstroduced previously 
master.solveModel(3600,0.1)


In [ ]:
#### PROBLEMA DE ALOCAÇÃO DE VEÍCULOS ####
class MasterProblem:
    def _init_(self, A_ijv, P_ijv, c_ijv, m_itv, d_ijt2, num_veiculos, num_pontos,num_periodos):
        self.A_ijv_GC = A_ijv
        self.P_ijv_GC = P_ijv
        self.c_ijv_GC = c_ijv
        self.m_itv_GC = m_itv
        self.d_ijt2_GC = d_ijt2
        self.m = Model("VAP")
        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
        
    def generateVariables(self):
        self.x_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)
        self.y_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)    
        
    def generateConstraints(self):
        #restrição de veículos
        self.m.addConstrs(quicksum(x_ijtv[(i,j,t,v)]+y_ijtv[(i,j,t,v)] for j in range(1,num_terminais)) 
             -quicksum(x_ijtv[(k,i,(t-tau_ij[k,i]),v)] + y_ijtv[(k,i,(t-tau_ij[k,i]),v)] for k in range(1,num_terminais+1) if k!=i and t>tau_ij[k,i])
             -y_ijtv[(i,i,(t-1),v)] == m_itv[(i,t,v)] for i,t,v in m_itv if t>1)
        #restrição da demanda
        self.m.addConstrs(quicksum(x_ijtv[(i,j,t,v)] for v in range(1,num_veiculos))<=d_ijt[(i,j,t)] for i,j,t in d_ijt)
        #restrição do percurso
        self.m.addConstrs(A_ijv[(i,j,v)]-x_ijtv[(i,j,t,v)]>=0 for i,j,t,v in x_ijtv)
        self.m.addConstrs(A_ijv[(i,j,v)]-y_ijtv[(i,j,t,v)]>=0 for i,j,t,v in y_ijtv)
    
    def generateObjective(self):
        self.m.setObjective(quicksum(p_ij[(i,j)]*x_ijtv[(i,j,t,v)]-c_ij[(i,j)]*y_ijtv[(i,j,t,v)] for i,j,t,v in aux_x_ijtv))
        
    def solveRelaxedModel(self):
        #relax integer variable to continuous variable
        self.relaxedModel = self.m.relax()
        self.relaxedModel.optimize()
        
    def getDuals(self):
        return self.relaxedModel.getAttr("P1", self.m.getConstrs())
    

In [ ]:
class DumpInitialAllocationGenerator:
    # Devolve os dicionários x e y com a solução inicial.
    def __init__():
        
    def generateBasicInitialPatterns(self):
        return (x,y)

In [ ]:
class Subproblem:
    def _init_(self, A_ijv, P_ijv, c_ijv, m_itv, d_ijt2, num_veiculos, num_pontos,num_periodos):
        self.A_ijv_GC = A_ijv
        self.P_ijv_GC = P_ijv
        self.c_ijv_GC = c_ijv
        self.m_itv_GC = m_itv
        self.d_ijt2_GC = d_ijt2
        self.m = Model("VAP")
        #u -> dual 
        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
        
    def generateVariables(self):
        self.x_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)
        self.y_ijtv = m.addVars(aux_x_ijtv, vtype=GRB.BINARY)    
        
    def generateConstraints(self):
        #restrição de veículos
        self.m.addConstrs(quicksum(x_ijtv[(i,j,t,v)]+y_ijtv[(i,j,t,v)] for j in range(1,num_terminais)) 
             -quicksum(x_ijtv[(k,i,(t-tau_ij[k,i]),v)] + y_ijtv[(k,i,(t-tau_ij[k,i]),v)] for k in range(1,num_terminais+1) if k!=i and t>tau_ij[k,i])
             -y_ijtv[(i,i,(t-1),v)] == m_itv[(i,t,v)] for i,t,v in m_itv if t>1)
        #restrição da demanda
        #self.m.addConstrs(quicksum(x_ijtv[(i,j,t,v)] for v in range(1,num_veiculos))<=d_ijt[(i,j,t)] for i,j,t in d_ijt)
        #restrição do percurso
        self.m.addConstrs(A_ijv[(i,j,v)]-x_ijtv[(i,j,t,v)]>=0 for i,j,t,v in x_ijtv)
        self.m.addConstrs(A_ijv[(i,j,v)]-y_ijtv[(i,j,t,v)]>=0 for i,j,t,v in y_ijtv)
    
    def generateObjective(self):
        self.m.setObjective(quicksum(p_ij[(i,j)]*x_ijtv[(i,j,t,v)]-c_ij[(i,j)]*y_ijtv[(i,j,t,v)] for i,j,t,v in aux_x_ijtv))
        
    

In [ ]:
#ordenando pelo tempo
m_sorted = m_it_0.sort_values(by=1, axis=1, ascending=True, kind='quicksort')
d_sorted = d_ijt_0.sort_values(by=2, axis=1, ascending=True, kind='quicksort')
veic_iv = pd.DataFrame() #m_itv.iloc[[0,2],:] #veículos no sistema


x = dict()
y = dict()

for t in range(d_ijt.shape[1]):
    if m_sorted.iloc[1,0]<=d_sorted.iloc[2,t]:# se há carros introduzidos antes da demanda
        if m_sorted.iloc[0,0] != d_sorted.iloc[0,t]:#se i são iguais
            x[(d_sorted.iloc[0,t],d_sorted.iloc[1,t],d_sorted.iloc[2,t],m_sorted.iloc[2,0])]
            veic_iv = pd.concat([veic_iv,m_itv.iloc[[0,2],0]])
        else:
            y[(m_sorted.iloc[0,0],d_sorted.iloc[0,t],d_sorted.iloc[2,t],m_sorted.iloc[2,0])]
                
        
    elif veic_iv[]
    
###################################

m_itv_1 = pd.DataFrame(m_itv_0)
d_ijt_1 = pd.DataFrame(d_ijt_0)

m = m_itv_1.sort_values(by=1, axis=1, ascending=True, kind='quicksort')
d = d_ijt_1.sort_values(by=2, axis=1, ascending=True, kind='quicksort')

veic_iv = pd.DataFrame()

for x in range(d.shape[1]):
    t = d.iloc[2,x]
    aux = 0
    while m.iloc[1,aux]==t: # atualiza lista de veículos no tempo t
         veic_iv = pd.concat([veic_iv,m_itv.iloc[[0,2],aux]])
    
    